In [1]:
workspace()
using Bolas
using Derivatives
using BolaIntervalo


    ^(Derive,Real) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Derivatives.jl:93
is ambiguous with: 
    ^(Any,Integer) at intfuncs.jl:86.
To fix, define 
    ^(Derive,Integer)
before the new definition.
    promote_type(Type{T<:Number},Type{Interval}) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Intervals.jl:467
is ambiguous with: 
    promote_type(Type{None},Type{T}) at promotion.jl:103.
To fix, define 
    promote_type(Type{None},Type{Interval})
before the new definition.
    promote_type(Type{T<:Number},Type{Interval}) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Intervals.jl:467
is ambiguous with: 
    promote_type(Type{T},Type{T}) at promotion.jl:101.
To fix, define 
    promote_type(Type{_<:Number},Type{_<:Number})
before the new definition.


In [14]:
A=Bola(4,1)
B=Bola(2,1)
N=Bola(0.9999942779541016,9.5367431640625e-6)

function wilkinson20(x)
    
    w=1
    
    for i=1:20
        w*=(x-i)
    end
    
    return(w)
end
        

wilkinson20 (generic function with 1 method)

In [8]:
function revisar_cercanas(A,B,tol) #revisa si una raíz dada está cerca de las demás
    
    for i = 1:length(B)
        
        if distancia(A,B[i]) <= tol
            return(true) 
        end
        
        
    end
    
    return(false)
end

function quitar_repetidas(A,tol) #remueve raices que esten mas cerca que una cierta tolerancia
   
    temp=Bola[]
    
    for i=1:length(A)
        
        if !(revisar_cercanas(A[i],temp,tol))
            push!(temp,A[i])
        end
        
    end
    
    return(temp)
end

function quitar_no_deseadas(F::Function,B,x) #Funcion para quitar bolas donde la derivada de la funcion se hace cero
    
    temp=Bola[]
    dF(x)=F(makex(x)).d
    
    for i=1:length(B)
        if !contiene(dF(B[i]),x)
            push!(temp,B[i])
        end
    end
    
    return(temp)
end

function quitar_no_utiles(F::Function,B,x) #Se encarga de quitar bolas que no contienen ceros de la funcion
    
    temp=Bola[]
    
    for i=1:length(B)
        if contiene(F(B[i]),x)
            push!(temp,B[i])
        end
    end
    
    return(temp)
end



function operador_de_newton_bola(F::Function,B::Bola) 
    
    dF(x)=F(makex(x)).d
    m=Bola(B.centro,1e-15) #Le asigno un radio mínimo al punto m 
    
    
    return (m-F(m)/(dF(B)))
    
end   

function metodo_newton_bola(F::Function, B::Bola)
    
    X=[B]
    
    i=0
    tolbis=1e-4
    tolradio=1e-15
    tolsimilar = 1e-2
    
    while(X[1].radio>tolbis)
        
       X=bisectar_bolas(X)
       X=quitar_no_utiles(F,X,0)
    
        
        i+=1
        
        if i==50
            break
        end
        
        X
 
    end
    
    @show X=quitar_no_deseadas(F,X,0)
    
    j=0
    
    radio_promedio=1
    
    while(tolradio<radio_promedio)
        
        radio_promedio=0
        
        
        for i=1:length(X)
            
            temp=operador_de_newton_bola(F,X[i])
            
            if interseccion_de_bolas(X[i],temp)==nothing
                X[i]=interseccion_de_bolas(B,temp)
            else
                X[i]=interseccion_de_bolas(X[i],temp)
            end
            
            radio_promedio+=(X[i].radio)
        end
         
        radio_promedio/=length(X)
        
        j+=1
        
        if j==50
            break
        end
    end
    
    X=quitar_repetidas(X,tolsimilar)
    
    for i=1:length(X) 
        println("Posible raíz en $(X[i].centro) ± $(X[i].radio)")
    end
    
end


f(x)=(x-1)*(x-2)*(x-3)*(x-4)*(x-5)*(x-6)*(x-7)
g(x)=(x+3)*(x-4)*(exp(x)-1)*cos(x)
M=metodo_newton_bola(f,Bola(6,12))

X = quitar_no_deseadas(F,X,0) => [Bola(1.000030517578125,9.1552734375e-5),Bola(1.999969482421875,9.1552734375e-5),Bola(2.999908447265625,9.1552734375e-5),Bola(3.000091552734375,9.1552734375e-5),Bola(4.000030517578125,9.1552734375e-5),Bola(4.999969482421875,9.1552734375e-5),Bola(5.999908447265625,9.1552734375e-5),Bola(6.000091552734375,9.1552734375e-5),Bola(7.000030517578125,9.1552734375e-5)]
Posible raíz en 1.000030517578125 ± 9.1552734375e-5
Posible raíz en 1.999969482421875 ± 9.1552734375e-5
Posible raíz en 2.9999999999999987 ± 4.440892098500626e-16
Posible raíz en 4.000000000000001 ± 8.881784197001252e-16
Posible raíz en 4.999999999999999 ± 8.881784197001252e-16
Posible raíz en 5.999908447265625 ± 9.1552734375e-5
Posible raíz en 7.000030517578125 ± 9.1552734375e-5


In [16]:
f(9)

40320

In [20]:
f(makex(Bola(17.000045776367188,7.62939453125e-5)))

Derive(Bola(-5.747022928343271e9,9.584469079784122e9),Bola(-1.2555454948726031e14,9.996361218228017e10))

In [31]:
operador_de_newton_bola(f,Bola(BigFloat(17.000045776367188),BigFloat(7.62939453125e-5)))

Bola(1.700000000325142377049077502723301958906439662258370670382541753535895760327455e+01 with 256 bits of precision,5.744931712241313052974049658227678358362202936489743807669440160575048763987605e+20 with 256 bits of precision)

In [3]:
M=[1,2,3,4,5,6,6,6,7,1]


for i=1:length(M)
    
    temp=Int[]
    
    for i=1:length(M)
        
        if !(M[i] in temp)
            push!(temp,M[i])
        end
    end
        
    return(temp)
end



7-element Array{Int64,1}:
 1
 2
 3
 4
 5
 6
 7

LoadError: M not defined
while loading In[2], in expression starting on line 30

In [62]:
M=[Bola(3),Bola(2),Bola(3),Bola(4),Bola(2),Bola(3.1)]

6-element Array{Bola,1}:
 Bola(3,1)  
 Bola(2,1)  
 Bola(3,1)  
 Bola(4,1)  
 Bola(2,1)  
 Bola(3.1,1)

In [50]:
function quitar_bolas_similares(B,tol::Real)
    
    return(temp)
end

quitar_bolas_similares (generic function with 1 method)

false

In [54]:
sort(M)

LoadError: `isless` has no method matching isless(::Bola, ::Bola)
while loading In[54], in expression starting on line 1

In [47]:
57847/1.16

49868.10344827586

In [48]:
150000/15

10000.0

In [106]:
M=Bola([1,1],1)
N=Bola([1,2],1)

Bola([1,2],1)

In [110]:
M-N

Bola([0,-1],2)

In [9]:
A=Bola(2,1)
B=Bola(3,2)

A/B

Bola(0.6666666666666666,6.333333333333334)

In [10]:
using Intervals

In [11]:
Interval(1,3)/Interval(1,5)

Interval(0.19999999999999998,3.0)